In [78]:
# Part 1

import re

input = open("input.txt", 'r').read()
# input = """seeds: 79 14 55 13

# seed-to-soil map:
# 50 98 2
# 52 50 48

# soil-to-fertilizer map:
# 0 15 37
# 37 52 2
# 39 0 15

# fertilizer-to-water map:
# 49 53 8
# 0 11 42
# 42 0 7
# 57 7 4

# water-to-light map:
# 88 18 7
# 18 25 70

# light-to-temperature map:
# 45 77 23
# 81 45 19
# 68 64 13

# temperature-to-humidity map:
# 0 69 1
# 1 0 69

# humidity-to-location map:
# 60 56 37
# 56 93 4"""

input_regex = re.compile(
    "^seeds:(?P<seeds>.*)"
    "seed-to-soil map:(?P<seed_to_soil>.*)"
    "soil-to-fertilizer map:(?P<soil_to_fertilizer>.*)"
    "fertilizer-to-water map:(?P<fertilizer_to_water>.*)"
    "water-to-light map:(?P<water_to_light>.*)"
    "light-to-temperature map:(?P<light_to_temperature>.*)"
    "temperature-to-humidity map:(?P<temperature_to_humidity>.*)"
    "humidity-to-location map:(?P<humidity_to_location>.*)$", 
    re.MULTILINE|re.DOTALL)

parsed_file = input_regex.match(input)

def transform(seeds, adj_map):
    new_seeds = [None for x in seeds]
    for line in adj_map.strip().split('\n'):
        to_val, from_val, size = map(int, line.split(' '))
        for x in range(len(seeds)):
            if new_seeds[x]:
                continue
            seed = seeds[x]
            if seed >= from_val and seed <= from_val + size - 1:
                new_seeds[x] = seed + to_val - from_val
        
    for x in range(len(new_seeds)):
        if not new_seeds[x]:
            new_seeds[x] = seeds[x]
            
    return new_seeds
    
seeds = list(map(int, parsed_file.group('seeds').strip().split(' ')))
seeds = transform(seeds, parsed_file.group('seed_to_soil'))
seeds = transform(seeds, parsed_file.group('soil_to_fertilizer'))
seeds = transform(seeds, parsed_file.group('fertilizer_to_water'))
seeds = transform(seeds, parsed_file.group('water_to_light'))
seeds = transform(seeds, parsed_file.group('light_to_temperature'))
seeds = transform(seeds, parsed_file.group('temperature_to_humidity'))
seeds = transform(seeds, parsed_file.group('humidity_to_location'))

print(min(seeds))

227653707


In [236]:
# Part 2

import re

input = open("input.txt", 'r').read()
# input = """seeds: 79 14 55 13

# seed-to-soil map:
# 50 98 2
# 52 50 48

# soil-to-fertilizer map:
# 0 15 37
# 37 52 2
# 39 0 15

# fertilizer-to-water map:
# 49 53 8
# 0 11 42
# 42 0 7
# 57 7 4

# water-to-light map:
# 88 18 7
# 18 25 70

# light-to-temperature map:
# 45 77 23
# 81 45 19
# 68 64 13

# temperature-to-humidity map:
# 0 69 1
# 1 0 69

# humidity-to-location map:
# 60 56 37
# 56 93 4"""

input_regex = re.compile(
    "^seeds:(?P<seeds>.*)"
    "seed-to-soil map:(?P<seed_to_soil>.*)"
    "soil-to-fertilizer map:(?P<soil_to_fertilizer>.*)"
    "fertilizer-to-water map:(?P<fertilizer_to_water>.*)"
    "water-to-light map:(?P<water_to_light>.*)"
    "light-to-temperature map:(?P<light_to_temperature>.*)"
    "temperature-to-humidity map:(?P<temperature_to_humidity>.*)"
    "humidity-to-location map:(?P<humidity_to_location>.*)$", 
    re.MULTILINE|re.DOTALL)

parsed_file = input_regex.match(input)

def process_seeds(seed_list):
    seed_groups = []
    
    while seed_list:
        start = seed_list[0]
        size = seed_list[1]
        seed_groups += [(start, start + size)]
        seed_list = seed_list[2:]
    
    return sorted(seed_groups, key=lambda x:x[0])

def transform(seed_groups, adj_map):
    processed_maps = []
    
    for line in adj_map.strip().split('\n'):
        to_val, from_val, size = map(int, line.split(' '))
        processed_maps += [(to_val, from_val, size)]
        
    processed_maps = sorted(processed_maps, key=lambda x:x[0])
    
    new_seed_groups = []
    not_checked_seed_groups = seed_groups
    
    while len(not_checked_seed_groups):
        seed_group = not_checked_seed_groups.pop()
        seed_start, seed_end = seed_group
        
        found = False
        for dest, source, size in processed_maps:
            new_seed_start = max(seed_start, source)
            new_seed_end = min(seed_end, source + size)
            if new_seed_start >= new_seed_end: # The seed_group doesn't overlap this translation rule
                continue
                
            # We match some part of this translation rule
            found = True
            
            if new_seed_start != seed_start and (seed_start, new_seed_start - 1) not in not_checked_seed_groups:
                not_checked_seed_groups.append((seed_start, new_seed_start - 1))
            if new_seed_end != seed_end and (source + size, seed_end) not in not_checked_seed_groups:
                not_checked_seed_groups.append((source + size, seed_end))
                
            new_seed_groups += [(new_seed_start + dest - source, new_seed_end + dest - source)]
            
        if not found:
            new_seed_groups.append(seed_group)
            
    for seed_group in not_checked_seed_groups:
        new_seed_groups += [seed_group]
                
    new_seed_groups = sorted(new_seed_groups, key=lambda x:x[0])
    
    merged_seed_groups = []
    while len(new_seed_groups):
        current_seed_group = new_seed_groups[0]
        new_seed_groups = new_seed_groups[1:]
        if not len(new_seed_groups):
            merged_seed_groups.append(current_seed_group)
            break
            
        if current_seed_group[1] + 1 >= new_seed_groups[0][0]:
            new_seed_groups = [(current_seed_group[0], new_seed_groups[0][1])] + new_seed_groups[1:]
        else:
            merged_seed_groups.append(current_seed_group)
            
    return merged_seed_groups

# Go in reverse finding humidities in order of location
seed_groups = process_seeds(list(map(int, parsed_file.group('seeds').strip().split(' '))))
print("Starting seed_to_soil")
seed_groups = transform(seed_groups, parsed_file.group('seed_to_soil'))
print("Starting soil_to_fertilizer")
seed_groups = transform(seed_groups, parsed_file.group('soil_to_fertilizer'))
print("Starting fertilizer_to_water")
seed_groups = transform(seed_groups, parsed_file.group('fertilizer_to_water'))
print("Starting water_to_light")
seed_groups = transform(seed_groups, parsed_file.group('water_to_light'))
print("Starting light_to_temperature")
seed_groups = transform(seed_groups, parsed_file.group('light_to_temperature'))
print("Starting temperature_to_humidity")
seed_groups = transform(seed_groups, parsed_file.group('temperature_to_humidity'))
print("Starting humidity_to_location")
seed_groups = transform(seed_groups, parsed_file.group('humidity_to_location'))

print(seed_groups[0][0])

Starting seed_to_soil
Starting soil_to_fertilizer
Starting fertilizer_to_water
Starting water_to_light
Starting light_to_temperature
Starting temperature_to_humidity
Starting humidity_to_location
78775051
